# DEEPMIND ALPHAZERO CHESS

## TODO



## TODO LATER
- Create controller (MDN-LSTM should be trainable)
- Link Controller - Monte Carlo Tree Search 
- Create Self play class
- Create Arena class

DEEP MIND OPEN ACCESS PAPER 

https://kstatic.googleusercontent.com/files/2f51b2a749a284c2e2dfa13911da965f4855092a179469aedd15fbe4efe8f8cbf9c515ef83ac03a6515fa990e6f85fd827dcd477845e806f23a17845072dc7bd

RAY ALPHA ZERO IMPLEMENTATION 

https://github.com/ray-project/ray/tree/master/rllib/contrib/alpha_zero

DUPLICATED MCTS IMPLEMENTATION

https://github.com/suragnair/alpha-zero-general/blob/master/MCTS.py

DISTRIBUTED IMPLEMENTATION

https://github.com/mokemokechicken/reversi-alpha-zero/blob/master/src/reversi_zero/lib/ggf.py

CHESS MOVES

https://www.ichess.net/blog/chess-pieces-moves/

BOARD REPRESENTATIONS

https://medium.com/datadriveninvestor/reconstructing-chess-positions-f195fd5944e

ALPHA ZERO EXPLANATION

https://nikcheerla.github.io/deeplearningschool/2018/01/01/AlphaZero-Explained/

TRANSFORMER NETWORK IMPLEMENTATION

https://pytorch.org/tutorials/beginner/transformer_tutorial.html

In [ ]:
import chess, gym, pickle, random, torch, math
import chess.svg as svg
import numpy as np
import pandas as pd
from copy import copy, deepcopy
import matplotlib.pyplot as plt
import torch.optim as optim
# plt.style.use('ggplot')
from scipy.ndimage.interpolation import shift
from copy import deepcopy
from colorama import init, Fore, Back, Style
from collections import Counter

from utilities import *
# import constants 
from constants import *
from models.vae import CNN_VAE, train_vae, Conv
from models.mdn import MDN_RNN, loss_function, clip_grad_norm_
from models.controller import Controller

# init(convert = True)
# REFERENCES
# Tensorflow implementation for Chess
# https://github.com/saurabhk7/chess-alpha-zero

# Pytorch implementation for Connect4
# https://github.com/plkmo/AlphaZero_Connect4/tree/master/src

In [ ]:
class Chess_Environment(gym.Env):
    """Chess Environment"""
    def __init__(self):
        self.board = chess.Board()
        self.white_pieces = ['P', 'N', 'B', 'R', 'Q', 'K']
        self.black_pieces = [piece.lower() for piece in self.white_pieces]
        self.x_coords, self.y_coords = np.meshgrid(list(range(0, 8)), list(range(0, 8)))
        self.x_coords = self.x_coords / 7
        self.y_coords = self.y_coords / 7
        self.state_size = self.observe()[0].shape
        self.init_action_decoder()
        self.whites_turn = True

    def reset(self):
        """Resets the environment"""
        self.board = chess.Board()
        
    def terminal_test(self):
        """Checks if the game is over"""
        return self.board.is_game_over(claim_draw = True)
    
    def result(self):
        """Gives the end game result"""

        result = self.board.result(claim_draw = True)
        if result == '1-0':
            return 1
        elif result == '0-1':
            return -1
        elif result == '1/2-1/2':
            return 0.5
        elif result == '*':
            return 0
        else:
            raise Exception('Invalid Result', result)
        
    def legal_actions(self):
        """Provides a list of legal actions in current state"""
        legal_actions = [str(legal_action) for legal_action in list(self.board.legal_moves)]
        return legal_actions
    
    def encode(self):
        """Encodes game state into a string"""
        board_ = self.board.piece_map()
            
        encoded = {
            'board' : board_,
            'turn' : self.board.turn,
            'legal_actions' : self.legal_actions()
        }
        
        return pickle.dumps(encoded)
    
    def decode(self, encoded):
        """Decodes string into game state and sets board and turn"""
        decoded = pickle.loads(encoded)
        self.board.set_piece_map(decoded['board'])
        if decoded['turn'] == False:
            self.board = self.board.mirror()
        self.board.turn = decoded['turn']

    def observe(self):
        """Create observation from the game state"""

        board_ = copy(self.board)
            
        board_ = np.ndarray.flatten(np.array(board_.__str__().split())).reshape(8, 8)
        
        black_pawns = np.isin(copy(board_), ['p']).astype(int)
        black_knights = np.isin(copy(board_), ['n']).astype(int)
        black_rooks = np.isin(copy(board_), ['r']).astype(int)
        black_bishops = np.isin(copy(board_), ['b']).astype(int)
        black_queen = np.isin(copy(board_), ['q']).astype(int)
        black_king = np.isin(copy(board_), ['k']).astype(int)
         
        white_pawns = np.isin(copy(board_), ['P']).astype(int)
        white_knights = np.isin(copy(board_), ['N']).astype(int)
        white_rooks = np.isin(copy(board_), ['R']).astype(int)
        white_bishops = np.isin(copy(board_), ['B']).astype(int)
        white_queen = np.isin(copy(board_), ['Q']).astype(int)
        white_king = np.isin(copy(board_), ['K']).astype(int)
        
        state = np.array([
            white_pawns,
            white_knights,
            white_rooks,
            white_bishops,
            white_queen,
            white_king,
            black_pawns,
            black_knights,
            black_rooks,
            black_bishops,
            black_queen,
            black_king
        ])
        
        return state, self.legal_actions()
    
    def step(self, action):
        """Perform a step in the environment"""
        try:
            self.board.push_uci(action)
        except ValueError:
            print(f"ACTION {action} LEGAL ACTIONS\n{self.legal_actions()}")
        self.board = self.board.mirror()
        if self.whites_turn:
            self.whites_turn = False
        else:
            self.whites_turn = True
        return self.observe()
    
    def move_board(self, move):
        """Moves the board positions as per the move"""

        char_to_int = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8} 

        int_to_char = {v: k for k, v in char_to_int.items()}

        encoded_board = [(char_to_int[pos[0]], int(pos[1])) for pos in np.ndarray.flatten(BOARD)]

        new_board = [tuple(map(sum, zip((char_to_int[pos[0]], int(pos[1])), move))) for pos in np.ndarray.flatten(BOARD)]
        moves = []
        for pos, new_pos in zip(np.ndarray.flatten(BOARD), new_board):
            try:
                if new_pos[1] > 8:
                    raise Exception()
                if move[2] is None:
                    move_ = f'{pos}{int_to_char[new_pos[0]]}{new_pos[1]}'
                else:
                    move_ = f'{pos}{int_to_char[new_pos[0]]}{new_pos[1]}{move[2]}'
            
                if '-' in move_:
                    raise Exception()
            
                if '0' in move_:
                    raise Exception()
                
            except Exception:
                move_ = 'XXXX'
            moves.append(move_)   
        return np.array(moves).reshape(8, 8)
    
    def init_action_decoder(self):
        """Initialize the decoder to decode the actions"""
        decoder = []
        for key in MOVES.keys():
            decoder_ = self.move_board(MOVES[key])
            decoder.append(decoder_)
#             print(f'Move {key}\n', new_board, '\n')
    
        self.decoder = np.array(decoder)
        self.action_size = self.decoder.shape
        
    def select_action(self, logits):
        """Decodes the output from the NN to legal actions""" 
        decoder_ = np.ndarray.flatten(self.decoder)
        logits_ = np.ndarray.flatten(logits)

        move_logits = [(decoder_[idx].lower(), logits_[idx]) for idx in range(len(logits_))]
        move_logits = dict(move_logits)
        
        legal_move_logits = {legal_action: move_logits[legal_action] for legal_action in self.legal_actions()}
        probabilities = list(legal_move_logits.values()) / sum(list(legal_move_logits.values()))

        action = random.choices(list(legal_move_logits.keys()), weights = probabilities, k = 1)[0]
        return action
    
    def render(self):
        """Render chess board"""
        chess.svg.board(board = self.board)  

env = Chess_Environment()
env.board

In [ ]:
class Agent:
    
    def __init__(self, decoder, action_size, vae, mdn, controller):
        self.action_size = action_size
        self.decoder = decoder
        self.vae = CNN_VAE(vae, None, 'Latest')
        self.mdn = MDN_RNN(mdn, None, 'Latest')
        params = {
            'hidden_size' : self.mdn.hidden_size,
            'z_size' : self.vae.z_size,
            'action_size' : action_size,
            'expansion_size': 1200
        }
        self.controller = Controller('Test', params, False)
        
        self.char_to_int = {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8} 
        self.char_to_int_promo = {'r': 1, 'b': 2, 'q': 3, 'k': 4, 'n': 5, '': 6} 
        
        moves_df = pd.DataFrame(list(zip(self.char_to_int.values(), self.char_to_int.keys())), columns = ['Ids', 'Labels'])
        promo_df = pd.DataFrame(list(zip(self.char_to_int_promo.values(), self.char_to_int_promo.keys())), columns = ['Ids', 'Labels'])
        self.move_actions_ohe = pd.get_dummies(moves_df['Ids'])
        self.promo_actions_ohe = pd.get_dummies(promo_df['Ids'])
    
    def select_action(self, logits, legal_actions):
        """Decodes the output from the NN to legal actions""" 
        decoder_ = np.ndarray.flatten(self.decoder)
        logits_ = np.ndarray.flatten(logits)

        move_logits = [(decoder_[idx].lower(), logits_[idx]) for idx in range(len(logits_))]
        move_logits = dict(move_logits)
        
        legal_move_logits = {legal_action: move_logits[legal_action] for legal_action in legal_actions}
        probabilities = list(legal_move_logits.values()) / sum(list(legal_move_logits.values()))
        
        legal_actions = list(legal_move_logits.keys())
        action = random.choices(list(legal_move_logits.keys()), weights = probabilities, k = 1)[0]
        
        action_probabilities = {legal_actions[idx] : probabilities[idx] for idx in range(len(legal_actions))}
        return action, action_probabilities
    
    def one_hot_encode_action(self, action):
        """One hot encodes the action"""
        if len(action) == 4:
            encoded_action = [
                self.move_actions_ohe.loc[
                self.char_to_int[action[0]] - 1].values, 
                self.move_actions_ohe.loc[int(action[1]) - 1].values, 
                self.move_actions_ohe.loc[self.char_to_int[action[2]] - 1].values, 
                self.move_actions_ohe.loc[int(action[3]) - 1].values,
                self.promo_actions_ohe.loc[self.char_to_int_promo[''] - 1].values
            ]
        else:
            encoded_action = [
                self.move_actions_ohe.loc[
                self.char_to_int[action[0]] - 1].values, 
                self.move_actions_ohe.loc[int(action[1]) - 1].values, 
                self.move_actions_ohe.loc[self.char_to_int[action[2]] - 1].values, 
                self.move_actions_ohe.loc[int(action[3]) - 1].values,
                self.promo_actions_ohe.loc[self.char_to_int_promo[action[4]] - 1].values
            ]
        ohe_action = np.concatenate(encoded_action)
        return ohe_action
    
    def reset(self, batch_size):
        """Resets the MDNs hidden state"""
        self.hidden = self.mdn.init_hidden(batch_size)
        
    def act(self, state, legal_actions):
        """Gets an action from the agent"""
        state = torch.tensor(state).float().unsqueeze(0)
        mu, logvar = self.vae.encode(state)
        z = self.vae.reparameterize(mu, logvar).squeeze(-1)
        zh = torch.cat((z, self.hidden[0].squeeze(0)), dim = 1)
        if zh.shape[0] > 1:
            raise Exception('Batch size > 1 not handled')
        logits, values = self.controller(zh)
        logits, value = logits.detach().squeeze(0).cpu().numpy(), values.detach().squeeze(0).cpu().numpy()[0]
        action, action_probabilities = self.select_action(logits, legal_actions)
        print(value)
        return action_probabilities, value
    
    def mcts_act(self, state, legal_actions):
        """Gets an action from the agent"""
        logits = np.random.rand(env.action_size[0], env.action_size[1], env.action_size[2])
        action, action_probabilities = self.select_action(logits, legal_actions)
        
        return action_probabilities, np.random.rand(10)[0]
    

agent = Agent(env.decoder, env.action_size, 'Test', 'Test', 'Test')
agent.reset(1)

#### Encode State

In [ ]:
# state, legal_actions = env.observe()

# state_planes = [
#     'White Pawns',
#     'White Knights',
#     'White Rooks',
#     'White Bishops',
#     'White Queen',
#     'White King',
#     'Black Pawns',
#     'Black Knights',
#     'Black Rooks',
#     'Black Bishops',
#     'Black Queen',
#     'Black King'
# ]

# for inx, state_ in enumerate(state):
#     fig = plt.figure()
#     fig.suptitle(state_planes[inx], fontsize = 20)
#     xticks = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
#     yticks = list(range(1, 9))
#     yticks.reverse()
#     plt.xticks(list(range(0, 8)), xticks)
#     plt.yticks(list(range(0, 8)), yticks)
#     plt.imshow(state_)


#### Encode Actions

In [ ]:
# queen_directions = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
# queen_distance = list(range(1, 8))
# queen_moves = [f'{direction}{distance}' for direction in queen_directions for distance in queen_distance]

# knight_moves = ['2N1E', '1N2E', '1S2E', '2S1E', '2S1W', '1S2W', '1N2W', '2N1W']
# underpromotion_moves = ['DOUBLEM', 'NECUT', 'NWCUT']
# moves = queen_moves + knight_moves + underpromotion_moves

# print(f'{len(queen_moves)} QUEEN MOVES {queen_moves[: 15]}\n{len(knight_moves)} KNIGHT MOVES {knight_moves}\n{len(underpromotion_moves)} UNDERPROMOTION MOVES {underpromotion_moves}')
# print(f'\nTOTAL MOVES {len(moves)}')
# print(moves)

#### Encode Actions

In [ ]:
# def move_board(move):
#     """Moves the board positions as per the move"""
#     print(move)
#     board = np.array([
#         ['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8'],
#         ['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7'],
#         ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6'],
#         ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5'],
#         ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4'],
#         ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3'],
#         ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2'],
#         ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1']
#     ])

#     char_to_int = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8} 

#     int_to_char = {v: k for k, v in char_to_int.items()}

#     encoded_board = [(char_to_int[pos[0]], int(pos[1])) for pos in np.ndarray.flatten(board)]

#     new_board = [tuple(map(sum, zip((char_to_int[pos[0]], int(pos[1])), move))) for pos in np.ndarray.flatten(board)]
 
#     moves = []
#     for pos, new_pos in zip(np.ndarray.flatten(board), new_board):
#         try:
# #             print(pos, new_pos)
#             if new_pos[1] > 8:
#                 raise Exception()
            
#             if move[2] is None:
#                 move_ = f'{pos}{int_to_char[new_pos[0]]}{new_pos[1]}'
#             else:
#                 move_ = f'{pos}{int_to_char[new_pos[0]]}{new_pos[1]}{move[2]}'
            
#             if '-' in move_:
#                 raise Exception()
            
#             if '0' in move_:
#                 raise Exception()
                
#         except Exception:
#             move_ = 'XXXX'
#         moves.append(move_)
#     return np.array(moves).reshape(8, 8)

# moves = {
#     # Queen Moves
#     'Q N1': (0, 1, None),
#     'Q N2': (0, 2, None),
#     'Q N3': (0, 3, None),
#     'Q N4': (0, 4, None),
#     'Q N5': (0, 5, None),
#     'Q N6': (0, 6, None),
#     'Q N7': (0, 7, None),
#     'Q NE1': (1, 1, None),
#     'Q NE2': (2, 2, None),
#     'Q NE3': (3, 3, None),
#     'Q NE4': (4, 4, None),
#     'Q NE5': (5, 5, None),
#     'Q NE6': (6, 6, None),
#     'Q NE7': (7, 7, None),
#     'Q E1': (1, 0, None),
#     'Q E2': (2, 0, None),
#     'Q E3': (3, 0, None),
#     'Q E4': (4, 0, None),
#     'Q E5': (5, 0, None),
#     'Q E6': (6, 0, None),
#     'Q E7': (7, 0, None),
#     'Q SE1': (1, -1, None),
#     'Q SE2': (2, -2, None),
#     'Q SE3': (3, -3, None),
#     'Q SE4': (4, -4, None),
#     'Q SE5': (5, -5, None),
#     'Q SE6': (6, -6, None),
#     'Q SE7': (7, -7, None),
#     'Q S1': (0, -1, None),
#     'Q S2': (0, -2, None),
#     'Q S3': (0, -3, None),
#     'Q S4': (0, -4, None),
#     'Q S5': (0, -5, None),
#     'Q S6': (0, -6, None),
#     'Q S7': (0, -7, None),
#     'Q SW1': (-1, -1, None),
#     'Q SW2': (-2, -2, None),
#     'Q SW3': (-3, -3, None),
#     'Q SW4': (-4, -4, None),
#     'Q SW5': (-5, -5, None),
#     'Q SW6': (-6, -6, None),
#     'Q SW7': (-7, -7, None),
#     'Q W1': (-1, 0, None),
#     'Q W2': (-2, 0, None),
#     'Q W3': (-3, 0, None),
#     'Q W4': (-4, 0, None),
#     'Q W5': (-5, 0, None),
#     'Q W6': (-6, 0, None),
#     'Q W7': (-7, 0, None),
#     'Q NW1': (-1, 1, None),
#     'Q NW2': (-2, 2, None),
#     'Q NW3': (-3, 3, None),
#     'Q NW4': (-4, 4, None),
#     'Q NW5': (-5, 5, None),
#     'Q NW6': (-6, 6, None),
#     'Q NW7': (-7, 7, None),
#     # Knight Moves
#     '2N1E' : (1, 2, None),
#     '1N2E' : (2, 1, None),
#     '1S2E' : (2, -1, None),
#     '2S1E' : (1, -2, None),
#     '2S1W' : (-1, -2, None),
#     '1S2W' : (-2, -1, None),
#     '1N2W' : (-2, 1, None),
#     '2N1W' : (-1, 2, None),
#     # Promtions
#     'PQ N1' : (0, 1, 'q'),
#     'PR N1' : (0, 1, 'r'),
#     'PB N1' : (0, 1, 'b'),
#     'PN N1' : (0, 1, 'n'),
#     'PQ NE1' : (1, 1, 'q'),
#     'PR NE1' : (1, 1, 'r'),
#     'PB NE1' : (1, 1, 'b'),
#     'PN NE1' : (1, 1, 'n'),
#     'PQ NW1' : (-1, 1, 'q'),
#     'PR NW1' : (-1, 1, 'r'),
#     'PB NW1' : (-1, 1, 'b'),
#     'PN NW1' : (-1, 1, 'n')
# }

# new_boards = []
# for key in moves.keys():
#     print(f'Move {key}')
#     new_board = move_board(moves[key])
#     new_boards.append(new_board)
#     print(new_board, '\n')
    
# new_boards = np.array(new_boards)
# new_boards.shape

#### Decode Actions

In [ ]:
# # logits = env.action
# # np.random.rand(len(moves), 8, 8)

# print(env.action_size)
# new_boards_ = np.ndarray.flatten(new_boards)
# logits_ = np.ndarray.flatten(logits)

# move_logits = [(new_boards_[idx].lower(), logits_[idx]) for idx in range(len(logits_))]
# move_logits = dict(move_logits)

# legal_actions = ['g1h3','g1f3','b1c3','b1a3','h2h3','g2g3','f2f3','e2e3','d2d3','c2c3','b2b3','a2a3','h2h4','g2g4','f2f4','e2e4','d2d4','c2c4','b2b4','a2a4']

# legal_move_logits = {legal_action: move_logits[legal_action] for legal_action in legal_actions}
# legal_move_logits

#### Create Training Data for VAE and figure out action size

In [ ]:
# def create_vae_training_data(games):
#     """Creates training date for CNN-VAE"""
#     actions = {}
#     states = {}
#     for game in range(1, games + 1):
#         env.reset()
#         state, legal_actions = env.observe()
#         encoded_state = env.encode()
#         states[encoded_state] = state
#         while not env.terminal_test():
#             action = random.choice(legal_actions)            
#             state, legal_actions = env.step(action)
#             encoded_state = env.encode()
#             states[encoded_state] = state
# #             actions[encoded_state] = agent.one_hot_encode_action(action)
#         print(f'Game {game} | Unique States {len(states.keys())} Unique Actions {len(actions)}', end = '\r')
    
#     validate_path('data/vae')
#     with open('data/vae/states.pkl', 'wb') as file:
#         pickle.dump(states, file)
#     with open('data/vae/actions.pkl', 'wb') as file:
#         pickle.dump(actions, file)
    
# create_vae_training_data(100)

#### Train VAE

In [ ]:
# def train_vae_(name, starting_channels, filename, z_size, epochs, batch_size):
#     """Trains the CNN-VAE"""
#     params = {
#         'z_size' : z_size,
#         'batch_size' : batch_size,
#         'learning_rate' : 1e-4,
#         'kl_tolerance' : 0.5,
#         'batch_norm' : False,
#         'starting_channels' : starting_channels
#     }

#     vae = CNN_VAE(name, params, False)
#     with open(f'data/vae/{filename}',  'rb') as pickle_file:
#         data = list(pickle.load(pickle_file).values())
#         data = [np.array(data_) for data_ in data]
    
#     train_vae(vae, data, epochs, 100)

    
# train_vae_('Test States', env.state_size[0], 'states.pkl', 600, 100, 200)
# # train_vae_('Test Actions', env.action_size[0], 'actions.pkl', 600, 100, 200)

#### Create Training Data for MDN

In [ ]:
# def create_mdn_training_data(name, games):
#     """Create training data for MDN-RNN"""
#     vae = CNN_VAE(name, None, 'Latest')
#     vae.eval()
#     data = pd.DataFrame(columns = ['Game ID', 'Sequence #', 'State ID'])
#     rollouts = []
#     transitions = 0
    
#     for game in range(1, games + 1):
#         states = []
#         actions = []
#         env.reset()
        
#         state, legal_actions = env.observe()
#         while not env.terminal_test():
#             action = random.choice(legal_actions)
#             states.append(state)
#             actions.append(agent.one_hot_encode_action(action))
#             state, legal_actions = env.step(action)
#             transitions += 1
            
#         mus, logvars = vae.encode(torch.tensor(states).float())
#         zs = vae.reparameterize(mus, logvars)
#         mus, logvars, zs = mus.squeeze(-1), logvars.squeeze(-1), zs.squeeze(-1)
#         actions = torch.tensor(actions).float().squeeze(-1)
#         zas = torch.cat([zs, actions], dim = 1)
#         rollouts.append((zs.detach().numpy(), actions.detach().numpy()))
        
#         print(f'Game {game} Transitions {transitions}', end = '\r')
    
#     validate_path('data/mdn')
#     with open('data/mdn/rollouts.pkl', 'wb') as file:
#         pickle.dump(rollouts, file)
    
    
# create_mdn_training_data('Test', 600)

In [ ]:
# def training_mdn(vae_name, name):
#     torch.cuda.empty_cache() 
#     """Trains the MDN-RNN"""
#     epochs = 499
#     batch_size = 5
    
#     params = {
#         'hidden_size' : 100,
#         'gaussian_size' : 20,
#         'stacked_layers': 1,
#         'grad_clip' : 0.5,
#         'learning_rate' : 1e-4
#     }
    
#     with open(f'data/mdn/rollouts.pkl', 'rb') as pickle_file:
#         rollouts = pickle.load(pickle_file)
    
#     params['z_size'] = rollouts[0][0].shape[1]
#     params['action_size'] = rollouts[0][1].shape[1]
#     params['batch_size'] = batch_size
    
#     mdn = MDN_RNN(name, params, False)
    
#     mdn.train()
#     mdn = mdn.to(mdn.device)
#     optimizer = optim.Adam(mdn.parameters(), lr = mdn.learning_rate)
        
#     inputs = []
#     targets = []
#     seq_lengths = []
    
#     for rollout in rollouts:
#         zs, actions = rollout
#         zas = np.concatenate((zs, actions), axis = 1)
#         seq_lengths.append(zas.shape[0])
#         inputs.append(zas)
#         targets.append(zs)
        
#     idxs = range(len(rollouts))
    
#     # Sorts order from longest to shortest sequence
#     idxs = [x for _, x in sorted(zip(seq_lengths, idxs), reverse = True)]
#     seq_lengths = [seq_lengths[x] for x in idxs]
#     inputs = [np.array(inputs[x]) for x in idxs]
#     targets = [np.array(targets[x]) for x in idxs]
    
#     max_seq_len = max(seq_lengths)
#     rollout_size = len(seq_lengths)
    
#     padded_X = np.zeros((rollout_size, max_seq_len, inputs[0].shape[-1]))
#     padded_Y = np.zeros((rollout_size, max_seq_len, targets[0].shape[-1]))
# #     https://towardsdatascience.com/taming-lstms-variable-sized-mini-batches-and-why-pytorch-is-good-for-your-health-61d35642972e
    
#     for i, x_len in enumerate(seq_lengths):
#         padded_X[i, 0 : x_len] = inputs[i]
#         padded_Y[i, 0 : x_len] = targets[i]
        
#     inputs = torch.tensor(padded_X).float().squeeze(1).to(mdn.device)
#     targets = torch.tensor(padded_Y).float().squeeze(1).to(mdn.device)
    
#     if inputs.shape[0] % batch_size != 0:
#         inputs = inputs[inputs.shape[0] // batch_size, :, :]
        
#     inputs = inputs.reshape(-1, batch_size, max_seq_len, mdn.z_size + mdn.action_size)
#     targets = targets.reshape(-1, batch_size, max_seq_len, mdn.z_size)
#     seq_lengths = list(torch.tensor(seq_lengths).reshape(-1, batch_size).numpy())
    
#     for epoch in range(epochs + 1):
#         for batch in range(inputs.shape[0]):
#             inputs_ = inputs[batch]
#             targets_ = targets[batch]
#             seq_lengths_ = list(seq_lengths[batch])
            
#             targets_ = torch.nn.utils.rnn.pack_padded_sequence(targets_, seq_lengths_, batch_first = True)
#             targets_, _ = torch.nn.utils.rnn.pad_packed_sequence(targets_, batch_first = True)
            
#             # Set initial hidden and cell states
#             hidden = mdn.init_hidden(batch_size)

#             # Forward pass
#             hidden = (hidden[0].detach(), hidden[1].detach())
#             (pi, mu, sigma), hidden = mdn(inputs_, hidden, seq_lengths_)
#             loss = loss_function(targets_, pi, mu, sigma)

#             # Backward and optimize
#             mdn.zero_grad()
#             loss.backward()
#             clip_grad_norm_(mdn.parameters(), mdn.grad_clip)
#             optimizer.step()
#         print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch, epochs, loss.item()))
            
#     mdn.save_model()
# training_mdn('Test', 'Test')

# ALPHA GO TIPS 


## MCTS
Each simulation proceeds by
selecting in each state s a move a with low visit count, high move probability and high value
(averaged over the leaf states of simulations that selected a from s) according to the current
neural network fθ. The search returns a vector π representing a probability distribution over
moves, either proportionally or greedily with respect to the visit counts at the root state.
The parameters θ of the deep neural network in AlphaZero are trained by self-play reinforcement learning, starting from randomly initialised parameters θ. Games are played by selecting
moves for both players by MCTS, at ∼ πt
. At the end of the game, the terminal position sT is
scored according to the rules of the game to compute the game outcome z: −1 for a loss, 0 for
a draw, and +1 for a win. The neural network parameters θ are updated so as to minimise the
error between the predicted outcome vt and the game outcome z, and to maximise the similarity
of the policy vector pt
to the search probabilities πt
. Specifically, the parameters θ are adjusted
by gradient descent on a loss function l that sums over mean-squared error and cross-entropy
losses respectively

(p, v) = fθ(s)                    

l = (z − v) 2 − π > log p + c||θ||2


In [ ]:
class MCTS():
    """Monte Carlo Tree Search Algorithm geared for Neural Networks"""

    def __init__(self, env, agent, mcts_simulations = 100, max_depth = 100, delta = 0.5):
        self.env = env
        self.agent = agent
        self.cpuct = 0.2    # WARNING BULLSHIT NUMBER!
        self.delta = delta  # value to prevent crash if no edges are visited
        self.mcts_simulations = mcts_simulations
        self.max_depth = max_depth
        
        self.Qsa = {}       # stores Q values for s, a (as defined in the paper)
        self.Nsa = {}       # stores # times edge s, a was visited
        self.Ns = {}        # stores # times board s was visited
        self.Ps = {}        # stores initial policy (returned by neural net)

        self.Es = {}        # stores victory result (1, 0, -1) ended for board s
        self.Vs = {}        # stores legal actions for board s

    def action_probabilities(self, encoded_state, temp = 1):
        """
        This function performs numMCTSSims simulations of MCTS starting from
        canonicalBoard.
        Returns:
            probs: a policy vector where the probability of the ith action is
                   proportional to Nsa[(s,a)]**(1./temp)
        """
        
        for runs in range(self.mcts_simulations):
            print(f"{Fore.BLUE}MCTS SIMULATION {runs + 1}{Style.RESET_ALL}")
            self.search(encoded_state, 0)
        
        s = encoded_state
        self.env.decode(encoded_state)
        legal_actions = self.env.legal_actions()
        counts = [self.Nsa[(s,a)] if (s,a) in self.Nsa else 0 for a in legal_actions]
        # counts array represent the number of time each action edge from your current state was traversed

        if temp == 0: # temprature is 0 representing taking the best action possible (greedy)
            bestA = np.argmax(counts) # bestA: best action number : argmax Returns the indices of the maximum values
            probs = [0] * len(counts)
            probs[bestA] = 1
            action_probs = {legal_actions[idx] : probs[idx] for idx in range(len(legal_actions))}
            return action_probs # returns the definite move(s) with same greedy reward, out of which one move HAS to be played
        
        print(f"{Fore.BLUE}COUNTS {sum(counts)} {counts}{Style.RESET_ALL}")
        
        # Handles frequent draw situation when MCTS fails to explore when the game is over resulting 0 counts causing div 0 error
        if sum(counts) == 0:
            counts = [1 for _ in counts]
            
        counts = [x ** (1. / temp) + 0.5 for x in counts]
        probs = [x / float(sum(counts)) for x in counts]
        action_probs = {legal_actions[idx] : probs[idx] for idx in range(len(legal_actions))}
        return action_probs #returns the probablity of different moves that CAN be played resulting in uniform distribution


    def search(self, encoded_state, depth):
        """
        This function performs one iteration of MCTS. It is recursively called
        till a leaf node is found. The action chosen at each node is one that
        has the maximum upper confidence bound as in the paper.
        Once a leaf node is found, the neural network is called to return an
        initial policy P and a value v for the state. This value is propogated
        up the search path. In case the leaf node is a terminal state, the
        outcome is propogated up the search path. The values of Ns, Nsa, Qsa are
        updated.
        NOTE: the return values are the negative of the value of the current
        state. This is done since v is in [-1,1] and if v is the value of a
        state for the current player, then its value is -v for the other player.
        Returns:
            v: the negative of the value of the current canonicalBoard
        """

        s = deepcopy(encoded_state)
        self.env.decode(encoded_state)
        print(f"{Fore.GREEN}SEARCH DEPTH {depth}{Style.RESET_ALL}")
        
        # Check if its an terminal state, -1 Opponent Won, 0 Game not Over, 1 Player Won
        if s not in self.Es:
            self.Es[s] = self.env.result()
        if self.Es[s] != 0 or self.env.terminal_test():
            print("SIMULATION OVER!", self.env.terminal_test(), -self.Es[s], self.env.board.result())
            return -self.Es[s]
        
        state, legal_actions = self.env.observe()
#         legal_actions = self.env.legal_actions()

        if s not in self.Ps: #if the current state 's' is not explored/expanded before n=0 by MCTS then create a new node and rollout
            self.Ps[s], v = self.agent.act(state, legal_actions)
            print("self.Ps[s] at Depth", depth, "\n", self.Ps[s])
            valids = legal_actions

            self.Vs[s] = valids 
            self.Ns[s] = 0
            print("VALUE", -v)
            return -v
        
        valids = self.Vs[s] #as already visited the valid moves array 'Vs' is already initialized
        cur_best = -float('inf')
        best_act = -1
        
        self.env.decode(encoded_state)
        legal_actions = self.env.legal_actions()
        # pick the action with the highest upper confidence bound
        
        agent_actions = list(self.Ps[s].keys())
        
        # Handles the occasional legal action that isn't an actual legal action
        # Pawn side cut even though there is no enemy piece in respective position
        print("XXXXX PRE LEGAL ACTIONS\n", legal_actions)
        if not set(agent_actions) == set(legal_actions):
            legal_actions = agent_actions
        print("XXXXX POST LEGAL ACTIONS\n", legal_actions)
        for a in legal_actions:
            if (s,a) in self.Qsa:
                u = self.Qsa[(s,a)] + self.cpuct * self.Ps[s][a] * math.sqrt(self.Ns[s]) / (1 + self.Nsa[(s,a)])
#                 print("In Qsa")
            else:
                self.Ps[s][a] 
                u = self.cpuct * self.Ps[s][a] * math.sqrt(self.Ns[s])     # Q = 0 ? : node exists but not explored as added and initilized during nnet phase
#                 print("Not In Qsa", self.Ps[s][a], self.Ns[s], math.sqrt(self.Ns[s]))
#             print("U", u)
            if math.isnan(u):
                u = 0
#                 print("Override U", u)
            if u > cur_best:
                cur_best = u
                best_act = a
#         print(f"{Fore.GREEN}BEST ACTION {best_act}{Style.RESET_ALL}")
        a = best_act

        _, _ = self.env.step(a)
        encoded_next_state = self.env.encode()

        v = self.search(encoded_next_state, depth + 1) # RECURSION until leaf node or terminal node is found
        
        self.env.decode(encoded_state)
        
        if (s,a) in self.Qsa:
            self.Qsa[(s,a)] = (self.Nsa[(s,a)] * self.Qsa[(s,a)] + v) / (self.Nsa[(s,a)] + 1) #update the Q Value
            self.Nsa[(s,a)] += 1 # increment number of visits to this node in MCTS
        else:
            self.Qsa[(s,a)] = v # initialize the new node
            self.Nsa[(s,a)] = 1

        self.Ns[s] += 1
        return -v

env.reset()
mcts_white = MCTS(deepcopy(env), deepcopy(agent), mcts_simulations = 200)
mcts_black = MCTS(deepcopy(env), deepcopy(agent), mcts_simulations = 200)
game_move = 0

while not env.terminal_test():
    print(f"{Fore.RED}GAME MOVE {game_move}{Style.RESET_ALL}")
    encoded_state = env.encode()
    if env.whites_turn: 
        action_probs = mcts_white.action_probabilities(encoded_state)
    else:
        action_probs = mcts_black.action_probabilities(encoded_state)
    action = random.choices(list(action_probs.keys()), weights = action_probs.values(), k = 1)[0]
    print(f"{Fore.CYAN}ACTUAL MOVE\n {action_probs}{Style.RESET_ALL}")
    _, _ = env.step(action)
    
    game_move += 1
#     try:
#         print(f"{Fore.RED}GAME MOVE {game_move}{Style.RESET_ALL}")
#         encoded_state = env.encode()
#         if env.whites_turn: 
#             action_probs = mcts_white.action_probabilities(encoded_state)
#         else:
#             action_probs = mcts_black.action_probabilities(encoded_state)
#         action = random.choices(list(action_probs.keys()), weights = action_probs.values(), k = 1)[0]
#         print(f"{Fore.CYAN}ACTUAL MOVE\n {action_probs}{Style.RESET_ALL}")
#         _, _ = env.step(action)
    
#         game_move += 1
#     except Exception as e:
#         print(f'{Fore.RED}ERROR {e}{Style.RESET_ALL}')
#         if env.whites_turn: 
#             print(mcts_white.env.encode(), "\n", encoded_state)
#         else:
#             print(mcts_black.env.encode(), "\n", encoded_state)
#         break
        
print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! GAME COMPLETED !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

In [ ]:
# env.decode(b'\x80\x03}q\x00(X\x05\x00\x00\x00boardq\x01}q\x02(K?cchess\nPiece\nq\x03)\x81q\x04}q\x05(X\n\x00\x00\x00piece_typeq\x06K\x04X\x05\x00\x00\x00colorq\x07\x89ubK=h\x03)\x81q\x08}q\t(h\x06K\x03h\x07\x89ubK<h\x03)\x81q\n}q\x0b(h\x06K\x06h\x07\x89ubK:h\x03)\x81q\x0c}q\r(h\x06K\x03h\x07\x89ubK8h\x03)\x81q\x0e}q\x0f(h\x06K\x04h\x07\x89ubK6h\x03)\x81q\x10}q\x11(h\x06K\x01h\x07\x89ubK4h\x03)\x81q\x12}q\x13(h\x06K\x01h\x07\x89ubK3h\x03)\x81q\x14}q\x15(h\x06K\x01h\x07\x89ubK1h\x03)\x81q\x16}q\x17(h\x06K\x01h\x07\x89ubK0h\x03)\x81q\x18}q\x19(h\x06K\x01h\x07\x89ubK/h\x03)\x81q\x1a}q\x1b(h\x06K\x01h\x07\x89ubK*h\x03)\x81q\x1c}q\x1d(h\x06K\x01h\x07\x89ubK(h\x03)\x81q\x1e}q\x1f(h\x06K\x02h\x07\x89ubK#h\x03)\x81q }q!(h\x06K\x02h\x07\x89ubK!h\x03)\x81q"}q#(h\x06K\x03h\x07\x88ubK\x1dh\x03)\x81q$}q%(h\x06K\x01h\x07\x88ubK\x1bh\x03)\x81q&}q\'(h\x06K\x01h\x07\x88ubK\x17h\x03)\x81q(}q)(h\x06K\x01h\x07\x88ubK\x16h\x03)\x81q*}q+(h\x06K\x01h\x07\x88ubK\x15h\x03)\x81q,}q-(h\x06K\x01h\x07\x89ubK\x12h\x03)\x81q.}q/(h\x06K\x01h\x07\x88ubK\x11h\x03)\x81q0}q1(h\x06K\x01h\x07\x88ubK\x10h\x03)\x81q2}q3(h\x06K\x05h\x07\x89ubK\x07h\x03)\x81q4}q5(h\x06K\x04h\x07\x88ubK\x06h\x03)\x81q6}q7(h\x06K\x02h\x07\x88ubK\x04h\x03)\x81q8}q9(h\x06K\x06h\x07\x88ubK\x02h\x03)\x81q:}q;(h\x06K\x03h\x07\x88ubK\x01h\x03)\x81q<}q=(h\x06K\x02h\x07\x88ubK\x00h\x03)\x81q>}q?(h\x06K\x04h\x07\x88ubuX\x04\x00\x00\x00turnq@\x88X\r\x00\x00\x00legal_actionsqA]qB(X\x04\x00\x00\x00b5c6qCX\x04\x00\x00\x00b5a6qDX\x04\x00\x00\x00b5c4qEX\x04\x00\x00\x00b5a4qFX\x04\x00\x00\x00b5d3qGX\x04\x00\x00\x00b5e2qHX\x04\x00\x00\x00b5f1qIX\x04\x00\x00\x00h1h2qJX\x04\x00\x00\x00g1f3qKX\x04\x00\x00\x00g1e2qLX\x04\x00\x00\x00e1f2qMX\x04\x00\x00\x00e1d2qNX\x04\x00\x00\x00e1f1qOX\x04\x00\x00\x00e1d1qPX\x04\x00\x00\x00c1e3qQX\x04\x00\x00\x00c1a3qRX\x04\x00\x00\x00c1d2qSX\x04\x00\x00\x00c1b2qTX\x04\x00\x00\x00b1a3qUX\x04\x00\x00\x00b1d2qVX\x04\x00\x00\x00a1a3qWX\x04\x00\x00\x00a1a2qXX\x04\x00\x00\x00f4f5qYX\x04\x00\x00\x00h3h4qZX\x04\x00\x00\x00g3g4q[X\x04\x00\x00\x00c3c4q\\X\x04\x00\x00\x00b3b4q]eu.')
# CURRENT MAIN BOARD
print("TERMINAL TEST", env.terminal_test(), env.result(), "WHITES TURN", env.whites_turn)
print(f"LEGAL ACTIONS\n{env.legal_actions()}")
env.board

In [ ]:
print("BLACK TERMINAL TEST", mcts_black.env.terminal_test(), mcts_black.env.result(), "MCTS BLACKS TURN", not mcts_black.env.whites_turn)
print(f"LEGAL ACTIONS\n{mcts_black.env.legal_actions()}")
mcts_black.env.board

In [ ]:
print("WHITE TERMINAL TEST", mcts_white.env.terminal_test(), mcts_white.env.result(), "MCTS WHITE TURN", mcts_white.env.whites_turn)
print(f"LEGAL ACTIONS\n{mcts_white.env.legal_actions()}")
mcts_white.env.board

In [ ]:
from models.base import Neural_Network
import torch.nn as nn

class Controller(Neural_Network):
    """A Controller using Fully Connected Layers"""

    def __init__(self, name, params, load_model):
        """Initialize an Neural Network model given a dictionary of parameters.

        Params
        =====
        * **name** (string) --- name of the model
        * **params** (dict-like) --- a dictionary of parameters
        """
        super(Controller, self).__init__(name, params, load_model)
        self.name = name
        self.type = 'Controller'
        if load_model != False:
            self.load_model(load_model)
        else:
            self.params = params
        self.z_size = self.params['z_size']
        self.hidden_size = self.params['hidden_size']
        self.action_size = self.params['action_size']
        self.device = self.get_device()
        
        self.fc1 = nn.Linear(self.z_size + self.hidden_size, self.params['expansion_size'])
        self.fc_v1 = nn.Linear(self.z_size + self.hidden_size, 100)
        self.fc_v2 = nn.Linear(100, 20)
        self.fc_v3 = nn.Linear(20, 1)
        
        self.decoder = nn.Sequential(
            Conv(self.params['expansion_size'], 256, 3, 1, 0, conv = nn.ConvTranspose2d, activation = nn.ReLU, batch_norm = False),
            Conv(256, 128, 3, 1, 0, conv = nn.ConvTranspose2d, activation = nn.ReLU, batch_norm = False),
            Conv(128, 64, 3, 1, 0, conv = nn.ConvTranspose2d, activation = nn.ReLU, batch_norm = False),
            Conv(64, self.action_size[0], 2, 1, 0, conv = nn.ConvTranspose2d, activation = nn.ReLU, batch_norm = False),
#             Conv(32, 3, 6, 2, 0, conv = nn.ConvTranspose2d, activation = nn.Sigmoid, batch_norm = False)
        )
        
        
#         self.fc1 = nn.Linear(self.z_size + self.hidden_size, 100)
#         self.fc2 = nn.Linear(100, self.action_size)
#         self.softmax = nn.Softmax(dim = 1)

        if load_model != False:
            self.load_state_dict(self.weights)
        
        print(self, "\n\n")

    def forward(self, x):
        xp = self.fc1(x)
#         print("XP FC1", xp.shape)
        xp = xp.reshape(xp.shape[0], -1, 1, 1)
#         print("XP RESHAPE", xp.shape)
        xp = self.decoder(xp)
#         print("XP DECODER", xp.shape)
    
        xv = self.fc_v1(x)
#         print("XV FC1", xv.shape)
        xv = self.fc_v2(xv)
#         print("XV FC2", xv.shape)
        xv = self.fc_v3(xv)
#         print("XV FC3", xv.shape)
        return xp, xv
    
vae = CNN_VAE('Test', None, 'Latest')
mdn = MDN_RNN('Test', None, 'Latest')
print(env.action_size)


params = {
    'hidden_size' : mdn.hidden_size,
    'z_size' : vae.z_size,
    'action_size' : env.action_size,
    'expansion_size': 1200
}



controller = Controller('Test', params, False)
# z = torch.tensor(np.random.rand(1, z_size)).float()
# hidden = torch.tensor(np.random.rand(1, hidden_size)).float()
# za = torch.cat((z, hidden), dim = 1)
# print(z.shape)
# print(hidden.shape)
# print(za.shape)
# controller(za)

hidden = mdn.init_hidden(1)
state, legal_actions = env.observe()
state = torch.tensor(state).float().unsqueeze(0)
print(state.shape)
mu, logvar = vae.encode(state)
z = vae.reparameterize(mu, logvar).squeeze(-1)
zh = torch.cat((z, hidden[0].squeeze(0)), dim = 1)
logits, values = controller(zh)

print(logits.shape, values.shape)
# action_probabilities = 

